# AutoFlight Prosperity I - Mission Feasibility

Chinese air taxi startup Autoflight (https://www.autoflight.com/) announces what it claims is the world’s longest eVTOL flight to date. Its electric flying cab, ‘Prosperity I’, allegedly covered a distance of 250 kilometres without stopping to charge. The flying taxi is scheduled for certification in 2025. <br>

The mission was recorded in the following youtube video: <br>
https://www.youtube.com/watch?v=2Xf1_lTcAN0

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime as datetime

In [12]:
class mission_segment():
    def __init__(self, name, t1, t2, dist, alt):
        """
        name = segment name
        t1 = start time (datetime object)
        t2 = end time (datetime object)
        dist = distance list ([start,end]) in km
        alt = altitude list ([start,end]) in m
        """
        self.name = name
        self.t1 = t1
        self.t2 = t2
        self.dt = (t2 - t1).total_seconds()
        self.dist = dist
        self.alt = alt
        self._calc_info()
    def _calc_info(self):
        if self.name == 'Takeoff':
            # average vertical speed (m/s)
            self.vy = (self.alt[1]-self.alt[0])/self.dt
        elif self.name == 'Forward Transition':
            # horizontal acceleration (m/s2) (manually input v_cruise=44)
            v_cruise = 44
            self.ax = (v_cruise)/self.dt
        elif self.name == 'Climb':
            # average vertical speed (m/s)
            self.vy = (self.alt[1]-self.alt[0])/self.dt
        elif self.name == 'Fixed Wing Cruise':
            # cruise speed (km/h)
            self.vx = (self.dist[1]-self.dist[0])/(self.dt/3600)
        elif self.name == 'Descend':
            # average vertical speed (m/s)
            self.vy = (self.alt[1]-self.alt[0])/self.dt
        elif self.name == 'Backward Transition':
            # horizontal acceleration (m/s2) (manually input v_cruise=44)
            v_cruise = 44
            self.ax = (-v_cruise)/self.dt
        elif self.name == 'Landing':
            # average vertical speed (m/s)
            self.vy = (self.alt[1]-self.alt[0])/self.dt

#### From the above youtube video, the following data were noted.

In [13]:
mission_segment_list = []

mission_segment_list.append(mission_segment(name = 'Takeoff',
                                            t1   = datetime(2023,2,23,0,0,0),
                                            t2   = datetime(2023,2,23,0,0,40),
                                            dist = [0.0, 0.0032],
                                            alt  = [0.0, 80.0]))

mission_segment_list.append(mission_segment(name = 'Forward Transition',
                                            t1   = datetime(2023,2,23,0,0,40),
                                            t2   = datetime(2023,2,23,0,1,0),
                                            dist = [0.0032, 0.488],
                                            alt  = [80.0, 80.0]))

mission_segment_list.append(mission_segment(name = 'Climb',
                                            t1   = datetime(2023,2,23,0,1,0),
                                            t2   = datetime(2023,2,23,0,2,30),
                                            dist = [0.488, 4.0],
                                            alt  = [80.0, 120.0]))

mission_segment_list.append(mission_segment(name = 'Fixed Wing Cruise',
                                            t1   = datetime(2023,2,23,0,2,30),
                                            t2   = datetime(2023,2,23,1,34,0),
                                            dist = [4.0, 245.7],
                                            alt  = [120.0, 120.0]))

mission_segment_list.append(mission_segment(name = 'Descend',
                                            t1   = datetime(2023,2,23,1,34,0),
                                            t2   = datetime(2023,2,23,1,35,27),
                                            dist = [245.7, 249.4],
                                            alt  = [120.0, 80.0]))

mission_segment_list.append(mission_segment(name = 'Backward Transition',
                                            t1   = datetime(2023,2,23,1,35,38),
                                            t2   = datetime(2023,2,23,1,36,13),
                                            dist = [249.4, 250.2],
                                            alt  = [80.0, 80.0]))

mission_segment_list.append(mission_segment(name = 'Landing',
                                            t1   = datetime(2023,2,23,1,36,13),
                                            t2   = datetime(2023,2,23,1,37,13),
                                            dist = [250.2, 250.3],
                                            alt  = [80.0, 0.0]))

#### The above mission is visualized as follows.

In [14]:
plot_list = []

dx = [10,23,9,0,-13,-24,-11]
dy = [0,0,0,5,0,0,0]

for i, seg in enumerate(mission_segment_list):
    go_scatter = go.Scatter(mode = 'lines+markers',
                            x = seg.dist,
                            y = seg.alt,
                            name = seg.name,
                            line = dict(color='blue'),
                            text = seg.name,
                            hovertemplate = '<extra></extra>dist = %{x} km<br>'+'alt   = %{y}.0 m')
    plot_list.append(go_scatter)
    go_text = go.Scatter(mode = 'text',
                     x = [0.5*(seg.dist[0]+seg.dist[1])+dx[i]],
                     y = [0.5*(seg.alt[0]+seg.alt[1])+dy[i]],
                     text=f'<b>{seg.name}</b>',
                     hoverinfo='none')
    plot_list.append(go_text)

# Takeoff segment
seg = mission_segment_list[0]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])+15],
                 y = [0.5*(seg.alt[0]+seg.alt[1])-15],
                 text=f'{seg.vy} m/s<br>vertical climb<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Forward transition segment
seg = mission_segment_list[1]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1]+150)],
                 y = [0.5*(seg.alt[0]+seg.alt[1])],
                 text=f'{seg.ax:.2f} m/s2<br>horizontal acceleration<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Climb
seg = mission_segment_list[2]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])+35],
                 y = [0.5*(seg.alt[0]+seg.alt[1])],
                 text=f'Vy = {seg.vy:.2f} m/s<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Cruise segment
seg = mission_segment_list[3]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])],
                 y = [0.5*(seg.alt[0]+seg.alt[1])-10],
                 text=f'{seg.dist[1]-seg.dist[0]} km cruise at {seg.vx:.2f} km/h<br>for {seg.dt/3600} hours',
                 hoverinfo='none')
plot_list.append(go_text)

# Descend
seg = mission_segment_list[4]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])-40],
                 y = [0.5*(seg.alt[0]+seg.alt[1])],
                 text=f'Vy = {seg.vy:.2f} m/s<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Backward transition segment
seg = mission_segment_list[5]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1]-150)],
                 y = [0.5*(seg.alt[0]+seg.alt[1])],
                 text=f'{seg.ax:.2f} m/s2<br>horizontal acceleration<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)

# Landing segment
seg = mission_segment_list[6]
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(seg.dist[0]+seg.dist[1])-20],
                 y = [0.5*(seg.alt[0]+seg.alt[1])-15],
                 text=f'{seg.vy:.2f} m/s<br>vertical descend<br>for {seg.dt} s',
                 hoverinfo='none')
plot_list.append(go_text)   

# Summary
duration = (mission_segment_list[-1].t2-mission_segment_list[0].t1).total_seconds()
hours = duration//3600
minutes = (duration % 3600)//60
seconds = duration % 60
go_text = go.Scatter(mode = 'text',
                 x = [0.5*(mission_segment_list[3].dist[0]+mission_segment_list[3].dist[1])],
                 y = [0.5*(mission_segment_list[3].alt[0]+mission_segment_list[3].alt[1])-80],
                 text=f'<b>Mission Summary</b><br>Date: {mission_segment_list[0].t1:%b %d, %Y}<br>Duration: {hours}hour, {minutes} mins, {seconds} secs',
                 hoverinfo='none')
plot_list.append(go_text) 

layout = go.Layout(title=go.layout.Title(text="Simplified Mission Profile",x=0.5),
                   xaxis={'title':'Distance [km]','range':[-11,261]},
                   yaxis={'title':'Altitude [m]','range':[-10,140]})

fig = go.Figure(data=plot_list, layout=layout)
fig.update_layout(showlegend=False)
fig.show()

## Prosperity I - Specifications and Performance Approximations

### General

The Prosperity I's specifications are listed below.<br>

<b>Propulsion</b>: Fully electric Lift+Cruise<br>
<b>Powertrain</b>: 10 lifting rotors + 3 propellers<br>
<b>Wingspan</b>: 12.8 m<br>
<b>Height</b>: 3.3 m<br><br>

The Prosperity I's performance is listed below.<br>

<b>Max range</b>: 241.402 km+<br>
<b>Cruise speed</b>: 209.215 km/h+:<br>
<b>Max payload</b>: 408.233 kg+<br>
<b>Hover noise</b>: ~70 dBA<br><br>

<b>source</b> = https://www.autoflight.com/en/products/

### Power and Energy Requirement

Based on the mission profile, the power and energy required can then be calculated.<br>

The power required for the vertical take-off and landing segments are:

\begin{equation*}
P_{VTOL} = \frac{1}{\eta_{mech}}\Big(\frac{W_{MTOW}v_{VTOL}}{2} + \frac{fW_{MTOW}}{FM}\Big)\sqrt {\frac{fW_{MTOW}}{2 \rho S_{disk}}}
\tag{1}
\end{equation*}<br>

where $\eta_{mech}$ is the mechanical efficiency, $W_{MTOW}$ is MTOW weight, $v_{VTOL}$ is the VTOL speed, $f$ is the adjusment for downwash on fuselage, $FM$ is the figure of merit, $S_{disk}$ is the disk area, and $\rho$ is the air density.

The power required for the cruise segment (if $C_{L} <= C_{L_{max}})$:

\begin{equation*}
P_{cruise} = \frac{D_{cruise}v_{cruise}}{\eta_{p}}
\tag{2a}
\end{equation*}<br>

where $D_{cruise}$ is the total drag during cruise, $v_{cruise}$ is the cruise speed, and $\eta_{p}$ is the propeller efficiency.

Given that there is a fixed maximum $C_{L}$, there are configurations in which the wing could not produce sufficient lift to maintain cruise. In those cases, the vertical lift system (rotor) has to operate to compensate for the lift deficit. So, the following equation calculates the power required for the cruise segment when $C_{L} > C_{L_{max}}$:

\begin{equation*}
P_{cruise} = \frac{v_{cruise}}{\eta_{p}\eta_{mech}} \Big(D + \frac{L_{deficit}}{4eqS_{disk}} \Big)
\tag{2b}
\end{equation*}<br>

where $L_{deficit}$ is the difference between the lift generated by the wing and the weight of the aircraft.

While the power required for the climb/descend (c/d) segment:

\begin{equation*}
P_{c/d} = \frac{v_{c/d}}{\eta_{p} \eta_{mech}} \Big(D_{c/d} + \frac{L_{deficit}}{4eqS_{disk}} + W_{MTOW} \sin \gamma \Big)
\tag{3}
\end{equation*}<br>

where $D_{c/d}$ is the total drag during climb/descend, $v_{c/d}$ is the climb/descend speed, $\eta_{p}$ is the propeller efficiency, $\eta_{mech}$ is the mechanical efficiency, and $\gamma$ is the climb/descend angle.

Energy required per segment is:

\begin{equation*}
E_{segment} = P_{segment}t_{segment}
\tag{4}
\end{equation*}<br>

where $P_{segment}$ is the power for each segment calculated in Eq (1)-(3) and $t_{segment}$ is the time for each segment.

The battery is assumed to be at its end-of-life state, in which it can only hold 80% of its original capacity. The total battery mass is thus calculated as:

\begin{equation*}
m_{total-battery} = \frac{\Sigma E_{segment}}{0.8\eta_{battery}\tilde{E}}
\tag{5}
\end{equation*}<br>

where $\tilde{E}$ is the battery energy density (Wh/kg). It is assumed that $\eta_{battery} = 0.93$; which is the representative of the efficiency of Lithium-ion batteries.

<b>source</b>: Hascaryo, R. W. and Merret, J. M., <em>Configuration-Independent Initial Sizing Method for UAM/eVTOL Vehicles</em>, AIAA Aviation 2020 Forum.


In [24]:
def calc_P_vtol(mtow, v_vtol, S_disk, eta_mech, f, FM, rho):
    """
    Calculate power required for VTOL (Watt)
    
    mtow: Maximum Take-Off Weight (kg)
    v_vtol: vertical take-off and landing speed (m/s)
    S_disk: disk area (m/s2)
    eta_mech: mechanical efficiency
    f: adjustment for downwash on fuselage
    FM: figure of merit
    rho: air density (kg/m3)
    """
    g = 9.81 #m/s2
    P_vtol = (1/eta_mech) * (0.5*mtow*g*v_vtol + f*mtow*g/FM) * np.sqrt((f*mtow*g)/(2*rho*S_disk))
    return P_vtol

def calc_P_cruise(D_cruise, v_cruise, eta_p):
    """
    Calculate power required for cruise (Watt) when CL <= CL_max
    
    D_cruise: drag during cruise (N)
    v_cruise: cruise speed (km/h)
    eta_p: propeller efficiency
    """
    # Convert to SI
    v_cruise = v_cruise * 1000 / 3600 #m/s
    
    P_cruise = D_cruise*v_cruise/eta_p
    
    return P_cruise

def calc_P_cruise_augmented(mtow, D_cruise, v_cruise, eta_mech, eta_p, rho, S_disk, CL):
    """
    Calculate power required for cruise (Watt) when CL > CL_max
    
    D_cruise: drag during cruise (N)
    v_cruise: cruise speed (km/h)
    eta_p: propeller efficiency
    """
    # Convert to SI
    v_cruise = v_cruise * 1000 / 3600 #m/s
    g = 9.81
    q = 0.5 * rho * v_cruise**2
    
    # Difference between weight and lift produced by wing (CL = CL_max = 0.7)
    L_deficit = (CL-0.7) * mtow * g
    
    P_cruise_aug = (v_cruise/(eta_p * eta_mech)) * (D + ((L_deficit)/(4*q*S_disk)))

    return P_cruise_aug
    
def calc_P_c_d(D_c_d, v_c_d, eta_p):
    """
    Calculate power required for climb/descend (Watt)
    
    D_c_d: drag during climb/descend (N)
    v_c_d: speed during climb/descend (m/s)
    eta_p: propeller efficiency
    """
    P_c_d = D_c_d*v_c_d/eta_p
    return P_c_d

def calc_m_battery(E_segment, E_tilde, eta_battery=0.93):
    """
    Calculate battery mass available for a specific segment
    
    E_segment: energy required for the segment (Wh)
    E_tilde: battery energy density (Wh/kg)
    eta_battery: battery efficiency (0.93 is for Li-ion battery)
    """
    m_battery = E_segment / (0.8*eta_battery*E_tilde)
    return m_battery

def calc_total_m_battery(E_segment, E_tilde, eta_battery=0.93):
    """
    E_segment_list: a list of energy required for each segment (Wh)
    E_tilde: battery energy density (Wh/kg)
    eta_battery: battery efficiency (0.93 is for Li-ion battery)
    """
    W_battery = 0
    for _, E_segment in enumerate(E_segment_list):
        m_total_battery += calc_m_battery(E_segment, E_tilde, eta_battery)
    
    return m_total_battery

### Cruise range equation for electric aircraft

The electric energy consumption of the aircraft during cruise is: <br><br>
\begin{equation*}
E_{cruise} = \tilde{E}m_{battery-cruise} 0.8 \eta_{battery} = W \frac{D}{L} R
\tag{6}
\end{equation*}<br>
where $\tilde{E}$ is the battery energy density, $m_{battery-cruise}$ is the battery mass available for cruise, $\eta_{battery}$ is the battery efficiency (0.93 for Li-ion), $W$ is the aircraft weight (mtow*g), $L$ is the lift force, $D$ is the drag force, and $R$ is the cruise range. Rewriting the equation for cruise range, it yields:<br><br>
\begin{equation*}
R = 0.8 \eta_{battery} \tilde{E} \frac{m_{battery-cruise}}{mtow} \frac{1}{g} \frac{L}{D}
\tag{7}
\end{equation*}<br>
where $mtow$ is the maximum take-off weight of the aircraft and $g$ is the gravity acceleration.

For reversed-engineering purpose, MTOW is set as a function:
\begin{equation*}
mtow = 0.8 \eta_{battery} \tilde{E} \frac{m_{battery-cruise}}{R} \frac{1}{g} \frac{L}{D}
\tag{7}
\end{equation*}<br>

<b>source</b>: Raymer, D.P., <em>Aircraft Design: A Conceptual Approach,</em> 6th ed.; AIAA: Reston, VA. USA, 2018.

In [16]:
# For doing reversed engineering purpose: mtow is set as a function
def calc_mtow(R, E_tilde, m_battery_cruise, L_by_D, eta_battery):
    """
    R: cruise range (km)
    E_tilde: battery energy density (Wh/kg)
    m_battery_cruise: battery mass availabe for cruise segment (kg)
    L_by_D: Lift-to-drag ratio
    eta_battery: battery efficiency
    """
    # Convert to SI
    R = R * 1000 # m
    E_tilde = E_tilde * 3600 # Ws/kg
    g = 9.81 # m/s^2
    
    mtow = 0.8 * eta_battery * E_tilde * m_battery_cruise * L_by_D / g / R
    
    return mtow

### Aerodynamics
A drag force is calculated using the following equations.<br><br>

\begin{equation*}
D = \frac{1}{2} \rho v^2 S C_{D}
\tag{8}
\end{equation*}<br>

\begin{equation*}
C_{D} = C_{D_{p}} + C_{D_{i}}
\tag{9}
\end{equation*}<br>

where $\rho$ is the air density, $v$ is the aircraft velocity, $S$ is the wing area, $C_{D_{p}}$ is the parasite drag, and $C_{D_{i}}$ is the induced drag.<br>

The induced drag coefficient value is expressed as the following equation, assuming an ideal elliptical lift distribution.<br><br>

\begin{equation*}
C_{D_{i}} = \frac{C_{L}^2}{\pi e AR}
\tag{10}
\end{equation*}<br>

\begin{equation*}
AR = \frac{b^2}{S}
\tag{11}
\end{equation*}<br>

where $C_{L}$ is the lift coefficient, $e$ is the Oswald span efficiency coefficient, $AR$ is the aspect ratio, and $b$ is the wing span.<br>

When cruising, the lift coefficient is as follows.<br><br>

\begin{equation*}
C_{L} = \frac{W}{\frac{1}{2} \rho v^2 S}
\tag{12}
\end{equation*}<br>

\begin{equation*}
e = 1.78(1-0.045AR^{0.68})-0.64
\tag{13}
\end{equation*}<br>

There should be a maximum lift coefficient for a given configuration. For Cessna 172 GA aircraft, $C_{L_{max}} = 0.6$ was set for cruise and $C_{L_{max}} = 0.7$ for climb. However, in order to accommodate for borderline plausible cases, these limits were not strict; instead, hard limits were set at $C_{L_{max}} = 0.7$ for cruise and $C_{L_{max}} = 0.75$ for climb.

Substituting $C_{L}$ in Eq(12) into Eq(10) yields:<br><br>

\begin{equation*}
C_{D_{i}} = \frac{1}{\pi e AR}\big(\frac{W}{\frac{1}{2}\rho v^2S}\big)^2
\tag{14}
\end{equation*}<br>

Substituting $C_{D_{i}}$ in the Eq(14) into Eq(9):<br><br>

\begin{equation*}
C_{D} = C_{D_{p}} + \frac{1}{\pi e AR}\big(\frac{W}{\frac{1}{2}\rho v^2S}\big)^2
\tag{15}
\end{equation*}<br>

Substituting $C_{D}$ in Eq(15) into Eq(8):<br><br>

\begin{equation*}
D = \frac{1}{2}\rho v^2S \big( C_{D_{p}} + \frac{1}{\pi e AR}\big(\frac{W}{\frac{1}{2}\rho v^2S}\big)^2 \big)
\tag{16}
\end{equation*}<br>

In order to calculate the total drag, it is necessary to calculate the parasite drag. The component build-up method is used to calculate the parasite drag. The parasite drag at subsonic speed is expressed by the following equation.<br><br>

\begin{equation*}
C_{D_{p}} = \frac{\Sigma\big(C_{f}FFQS_{wet}\big)}{S_{ref}} + C_{D_{misc}} + C_{D_{L\&P}} 
\tag{17}
\end{equation*}<br>

where $C_{f}$ is the surface friction coefficient, $FF$ is the form factor, $Q$ is the interference coefficient, $S_{wet}$ is the wetted area, $S_{ref}$ is the reference area, $C_{D_{misc}}$ is the miscellaneous drag, and $C_{D_{L\&P}}$ is the leakages and protuberances drag coefficient.<br>

The surface friction coefficient of the plate is expressed by the following equation for laminar and turbulent flows.<br>

Laminar:
\begin{equation*}
C_{f} = \frac{1.328}{\sqrt{R}}
\tag{18}
\end{equation*}<br>

Turbulent:
\begin{equation*}
C_{f} = \frac{0.455}{(log_{10}R)^{2.58}(1+0.144M^2)^{0.65}}
\tag{19}
\end{equation*}<br>

where $R$ is the Reynolds number and $M$ is the Mach number.

<b>source</b>: Raymer, D.P., <em>Aircraft Design: A Conceptual Approach,</em> 6th ed.; AIAA: Reston, VA. USA, 2018.

In [17]:
# Aerodynamic models

def calc_CL_cruise(mtow, v_cruise, rho, S):
    """
    Calculate CL at cruise (assuming lift = weight)
    
    mtow: maximum take-off weight (kg)
    v_cruise: cruise speed (km/h)
    rho: air density
    S: wing reference area (m2)
    """
    # Convert into SI
    v_cruise = v_cruise * 1000 /3600 #m/s
    g = 9.81 #m/s2
    CL_cruise = (mtow*g)/(0.5*rho*v_cruise*v_cruise*S)
    
    return CL_cruise

def calc_CD_induced(CL, AR):
    """
    Calculate CD due to induced drag
    
    CL: lift coefficient
    AR: wing aspect ratio
    """
    e = 1.78 * (1 - 0.045 * AR**0.68) - 0.64 # oswald coeff
    CD_i = CL*CL/(np.pi * e * AR)
    
    return CD_i

def calc_CD_parasite(mtow, v_cruise, S, rho=1.17, K=0.15):
    """ 
    Calculate CD due to parasitic drag
    
    mtow: maximum take-off weight
    v_cruise: cruise speed km/h
    S: wing area
    
    ref: teTra's World eVTOL data
    https://docs.google.com/spreadsheets/d/1Xp6tqbU8aNPZ0nnScBRngmlGMr2DButvgyRbdxGNzvs/edit#gid=651362802
    """
    # Convert into SI
    g = 9.81 #m/s2
    v_cruise = v_cruise * 1000 / 3600
    
    CD_p = K * (2*mtow*g/S/rho)**2 / v_cruise**4
    
    return CD_p

In [18]:
# Example to calculate aero performance
mtow = 1500 #kg
rho = 1.17 # kg/m3 air density
v_cruise = 158 # km/h
b = 12.8 # wing span (m)
c = 1.5 # wing chord (m) (wing_chord < 1.5 is not possible for mtow=1500kg)
S = b*c
AR = b*b/S

CL = calc_CL_cruise(mtow, v_cruise, rho, S)
CD_i = calc_CD_induced(CL, AR)
CD_p = calc_CD_parasite(mtow, v_cruise, S, rho, K=0.15)
CD = CD_i + CD_p

print(f'Lift coeff: {CL}')
print(f'Total drag coeff: {CD}')

Lift coeff: 0.6801332380165901
Total drag coeff: 0.0910696084144719


### Iterative Sizing Equation

This following iterative process is similar to the methods developed for fuel-burning aircraft, however not exactly the same. It is because the methods for fuel-burning aircraft assumes that the aircraft gets lighter as the mission segment progresses, which is not the case for electric aircraft (the weight stays the same throughout the flight). The concept of <b>battery mass fraction</b> is introduced here.

Battery Mass Fraction (BMF) is the ratio of battery mass to all-up aircraft mass. The BMF values calculated for the various mission segments are used in an iterative process described in Raymer's book in which the fraction of total aircraft mass availabe for batteries is compared to the battery mass fraction required to do the mission. The BMF for a segment is defined as:

\begin{equation*}
BMF_{segment} = \frac{m_{battery-segment}}{mtow} = \frac{1}{mtow}\frac{P_{segment}t_{segment}}{0.8\eta_{battery}\tilde{E}}
\tag{20}
\end{equation*}<br>

where $m_{battery-segment}$ is the required battery mass for that segment (Watt), $P_{segment}$ is the the power required for that segment, $t_{segment}$ is the segment run-time (hour), $\eta_{battery}$ is the battery efficiency (0.93 for Li-ion), $\tilde{E}$ is the battery energy density (Wh/kg), and $mtow$ is in kg.

The available BMF is thus:<br>

\begin{equation*}
BMF_{available} = \frac{mtow - m_{empty} - m_{payload}}{mtow}
\tag{21}
\end{equation*}<br>

Solving for <b>mtow</b> yields the following sizing equation:<br>
\begin{equation*}
mtow = \frac{m_{payload}}{1- BMF - \frac{m_{empty}}{mtow}}
\tag{21}
\end{equation*}<br>


In [76]:
def BMF_segment(mtow, P_segment, t_segment, E_tilde, eta_battery=0.93):
    """
    Calculate BMF for a segment
    
    mtow: max takeoff weight (kg)
    P_segment: required power for a segment (Watt)
    t_segment: run-time for a segment (hour)
    E_tilde: battery energy density (Wh/kg)
    eta_battery: battery efficiency (0.93 for Li-ion)
    """
    BMF = (P_segment * t_segment)/(0.8 * eta_battery * E_tilde * mtow)
    
    return BMF

In [66]:
def calc_iterative_mtow(mtow_guess, m_payload, m_fuel_by_mtow):
    error = 100 #%
    n_iter = 1
    max_iter = 100
    while(error > 0.01 and n_iter < max_iter):
        m_empty_by_mtow = 0.97 * mtow_guess**(-0.06) 
        mtow_calc = (m_payload)/(1 - m_fuel_by_mtow - m_empty_by_mtow)
        error = abs(mtow_guess - mtow_calc)/mtow_calc * 100 #%
        print('--------------------------------------')
        print('| mtow_g | mtow_calc | error (%) |')
        print('--------------------------------------')
        print(f'| {mtow_guess:.2f} | {mtow_calc:.2f} |  {error:.2f}   |')
        mtow_guess = mtow_calc
        n_iter += 1
    return mtow_calc

m_payload = 2180+59102
mtow_calc = calc_iterative_mtow(77350, m_payload, 0.490797)
print('\n')
print(f'mtow = {mtow_calc}')

--------------------------------------
| mtow_g | mtow_calc | error (%) |
--------------------------------------
| 77350.00 | 3953198.73 |  98.04   |
--------------------------------------
| mtow_g | mtow_calc | error (%) |
--------------------------------------
| 3953198.73 | 513672.99 |  669.59   |
--------------------------------------
| mtow_g | mtow_calc | error (%) |
--------------------------------------
| 513672.99 | 894422.68 |  42.57   |
--------------------------------------
| mtow_g | mtow_calc | error (%) |
--------------------------------------
| 894422.68 | 738884.84 |  21.05   |
--------------------------------------
| mtow_g | mtow_calc | error (%) |
--------------------------------------
| 738884.84 | 785420.28 |  5.92   |
--------------------------------------
| mtow_g | mtow_calc | error (%) |
--------------------------------------
| 785420.28 | 769858.13 |  2.02   |
--------------------------------------
| mtow_g | mtow_calc | error (%) |
--------------------------

In [65]:
def calc_iterative_mtow(mtow_guess, m_payload, BMF):
    error = 100 #%
    n_iter = 1
    max_iter = 100
    while(error > 0.01 and n_iter < max_iter):
        m_empty_by_mtow = 0.97 * mtow_guess**(-0.06) 
        mtow_calc = (m_payload)/(1 - m_fuel_by_mtow - m_empty_by_mtow)
        error = abs(mtow_guess - mtow_calc)/mtow_calc * 100 #%
        print('--------------------------------------')
        print('| mtow_g | mtow_calc | error (%) |')
        print('--------------------------------------')
        print(f'| {mtow_guess:.2f} | {mtow_calc:.2f} |  {error:.2f}   |')
        mtow_guess = mtow_calc
        n_iter += 1
    return mtow_calc

### Trade Study

The following trade study is conducted based on the Autoflight - Prosperity I's mission profile and specifications. The following trade study is done:
1. MTOW vs Battery Energy Density (with different cruise ranges)
2. MTOW vs Cruise Range (with different battery energy densities)
3. MTOW vs Wing Area (with different battery energy densities)
4. MTOW vs Disk Area (with different battery energy densities)

#### 1. MTOW vs Battery Energy Density (with different cruise ranges)

In [75]:
# Cruise range (km)
range_list = [100, 150, 200, 250]

# Battery Energy Density (Wh/kg)
battery_list = [200, 220, 240, 260, 280, 300, 320]

# Assumptions
eta_battery = 0.93 # Li-ion is used
eta_p = 0.7 # propeller's efficiency, In Roskam's book, general aviation = 0.8, homebuilt aircraft = 0.7
rho = 1.17 # air density (kg/m2)

# Battery, range, cruise
E_tilde = range_list[0]
R = range_list[3]
v_cruise = 158 #km/h

# Wing (autoflight prosperity I)
wing_span = 12.8
wing_chord = 1.5

S = wing_span * wing_chord
AR = wing_span**2 / S

# --- iterative sizing procedure --- #
mtow = 2040 # initial guess

# Aerodynamics
CL = calc_CL_cruise(mtow, v_cruise, rho, S)
CD_i = calc_CD_induced(CL, AR)
CD_p = calc_CD_parasite(mtow, v_cruise, S, rho, K=0.15)
CD = CD_i + CD_p
L_by_D = CL/CD

D_cruise = CD * 0.5 * rho * (v_cruise*1000/3600)**2 * S

P_cruise_segment = calc_P_cruise(D_cruise, v_cruise, eta_p, CL)

E_cruise_segment = P_cruise_segment * (R/v_cruise)

m_battery_cruise = calc_m_battery(E_cruise_segment, E_tilde, eta_battery)

mtow = calc_mtow(R, E_tilde, m_battery_cruise, L_by_D, eta_battery)

mtow

0.9249812037025626


2914.2857142857147

#### 2. MTOW vs Cruise Range (with different battery energy densities)

#### 3. MTOW vs Wing Area (with different battery energy densities)

#### 4. MTOW vs Disk Area (with different battery energy densities)